In [1]:
from data_tools import get_full_text_data, get_data_with_dates, get_all_data

import pandas as pd
from tqdm.notebook import tqdm
from warnings import warn

In [2]:
dates = get_data_with_dates(get_full_text_data())

/home/proto/repos/crisis-detector/data_tools.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['path'] = data1['Nazwa pliku'].apply(lambda x: files[fnames.index(x)])


In [3]:
df = pd.DataFrame(columns=['topic', 'id', 'text'])
for row in tqdm(dates.itertuples(), total=len(dates)):
    try:
        src_df = pd.read_excel(row.path)
        new_df = pd.DataFrame(columns=['topic', 'id', 'text'])
        new_df['id'] = src_df.index
        new_df['text'] = src_df.apply(lambda x: " ".join([*str(x['Lead']).splitlines(), *str(x['OCR']).splitlines()]) + '\n', axis=1).tolist()
        new_df['topic'] = row.name
        new_df['sentiment'] = src_df['Sentyment']
        df = pd.concat((df, new_df), ignore_index=True)
    except KeyError:
        print(f'Missing column in {row.name}. Skipping...')

df.to_feather('saved_objects/full_text_df.feather')

  0%|          | 0/43 [00:00<?, ?it/s]

In [4]:
df = pd.read_feather('saved_objects/full_text_df.feather')
with open('saved_objects/texts_full.txt', 'w') as f:
    f.writelines(df['text'].to_list())

In [4]:
df = pd.read_feather('saved_objects/full_text_df.feather')
for i, text in tqdm(enumerate(df['text']), total=len(df)):
    with open('saved_objects/texts/' + str(i) + '.txt', 'w') as f:
        f.write(text)

  0%|          | 0/326885 [00:00<?, ?it/s]